In [1]:
import pandas as pd
import numpy as np
import zipfile as zf
from matplotlib import pyplot as plt

In [3]:
from sagemaker import get_execution_role

bucket_name = 'sagemaker-project'

role = get_execution_role()
bucket_key_prefix = 'cuisine-prediction'
vocabulary_length = 9013

print(role)

arn:aws:iam::264325520404:role/service-role/AmazonSageMaker-ExecutionRole-20220512T142894


In [ ]:
import boto3

s3 = boto3.resource('s3')
target_bucket = s3.Bucket(bucket_name)

output_path = 's3://{0}/{1}/output'.format(bucket_name, bucket_key_prefix)
code_location = 's3://{0}/{1}/code'.format(bucket_name, bucket_key_prefix)

In [ ]:
test_data = pd.read_csv('s3://{0}/'.format(bucket_name,"test_data.csv")

In [ ]:
test_data.drop("cuisine",inplace=True,axis=1)

In [ ]:
train_data = pd.read_csv('s3://{0}/'.format(bucket_name,"train_data.csv")

In [ ]:
train_data

,recipe_id,cuisine,( oz.) tomato sauce,(14.5 oz.) diced tomatoes,1% low-fat buttermilk,1% low-fat cottage cheese,1% low-fat milk,2% low-fat cottage cheese,2% lowfat greek yogurt,2% reduced-fat milk,...,yucca root,yukon gold,yukon gold potatoes,yuzu,za'atar,zest,zesty italian dressing,zinfandel,ziti,zucchini
0,0,spanish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,mexican,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,french,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,chinese,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,italian,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39766,49712,indian,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39767,49713,southern_us,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39768,49714,irish,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39769,49716,indian,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_data = train_data.reset_index()
train_data = train_data.fillna(0)

In [ ]:
train_data.columns

Index(['index', 'recipe_id', 'cuisine', '(    oz.) tomato sauce',
       '(14.5 oz.) diced tomatoes', '1% low-fat buttermilk',
       '1% low-fat cottage cheese', '1% low-fat milk',
       '2% low-fat cottage cheese', '2% lowfat greek yogurt',
       ...
       'yucca root', 'yukon gold', 'yukon gold potatoes', 'yuzu', 'za'atar',
       'zest', 'zesty italian dressing', 'zinfandel', 'ziti', 'zucchini'],
      dtype='object', length=4295)

In [ ]:
train_X = train_data.drop(['index','cuisine','recipe_id'], axis=1).values
train_X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
train_X.shape

(39771, 4292)

In [ ]:
train_data['cuisine'] = train_data['cuisine'].map(
    { 
        'greek':0, 
        'southern_us':1, 
        'filipino':2, 
        'indian':3, 
        'jamaican':4,
        'spanish':5, 
        'italian':6, 
        'mexican':7, 
        'chinese':8, 
        'british':9, 
        'thai':10,
        'vietnamese':11, 
        'cajun_creole':12, 
        'brazilian':13, 
        'french':14, 
        'japanese':15,
        'irish':16, 
        'korean':17, 
        'moroccan':18, 
        'russian':19
    }
)

train_Y = train_data['cuisine'].values
train_Y

array([ 5,  7, 14, ..., 16,  3,  7])

In [ ]:
import tensorflow.keras as tf

model = tf.models.Sequential()


model.add(tf.layers.Dense(768, activation='relu'))
#model.add(tf.layers.Dropout(0.5))

#model.add(tf.layers.Dense(2048, activation='relu'))
#model.add(tf.layers.Dropout(0.5))
#model.add(tf.layers.Dense(256, activation='relu'))
#model.add(tf.layers.Dense(512, activation='relu'))
#model.add(tf.layers.Dropout(0.5))
#model.add(tf.layers.Dense(1024, activation='relu'))

model.add(tf.layers.Dense(256, activation='relu'))
#model.add(tf.layers.BatchNormalization())
#model.add(tf.layers.Dropout(0.5))


model.add(tf.layers.Dense(20, activation='softmax'))
model.compile(optimizer='adam'
              ,loss='sparse_categorical_crossentropy'
              ,metrics=['accuracy']
             )

#model.summary()

res = model.fit(train_X, train_Y, epochs=10, validation_split=0.1)

Epoch 1/10
1119/1119 [==============================] - 10s 7ms/step - loss: 0.9478 - accuracy: 0.7246 - val_loss: 0.7428 - val_accuracy: 0.7861
Epoch 2/10
1119/1119 [==============================] - 7s 6ms/step - loss: 0.4824 - accuracy: 0.8513 - val_loss: 0.7476 - val_accuracy: 0.7841
Epoch 3/10
1119/1119 [==============================] - 7s 6ms/step - loss: 0.2598 - accuracy: 0.9180 - val_loss: 0.8345 - val_accuracy: 0.7891
Epoch 4/10
1119/1119 [==============================] - 6s 6ms/step - loss: 0.1200 - accuracy: 0.9629 - val_loss: 0.9927 - val_accuracy: 0.7831
Epoch 5/10
1119/1119 [==============================] - 7s 6ms/step - loss: 0.0559 - accuracy: 0.9835 - val_loss: 1.1969 - val_accuracy: 0.7818
Epoch 6/10
1119/1119 [==============================] - 6s 6ms/step - loss: 0.0356 - accuracy: 0.9898 - val_loss: 1.3357 - val_accuracy: 0.7831
Epoch 7/10
1119/1119 [==============================] - 7s 6ms/step - loss: 0.0257 - accuracy: 0.9929 - val_loss: 1.5128 - val_accuracy

In [ ]:
test_data = test_data.reset_index()
test_data = test_data.fillna(0)
test_data.columns

Index(['index', 'recipe_id', '(    oz.) tomato sauce',
       '(14.5 oz.) diced tomatoes', '1% low-fat buttermilk',
       '1% low-fat cottage cheese', '1% low-fat milk',
       '2% low-fat cottage cheese', '2% lowfat greek yogurt',
       '2% reduced-fat milk',
       ...
       'yucca root', 'yukon gold', 'yukon gold potatoes', 'yuzu', 'za'atar',
       'zest', 'zesty italian dressing', 'zinfandel', 'ziti', 'zucchini'],
      dtype='object', length=4294)

In [ ]:
train_data.columns

Index(['index', 'recipe_id', 'cuisine', '(    oz.) tomato sauce',
       '(14.5 oz.) diced tomatoes', '1% low-fat buttermilk',
       '1% low-fat cottage cheese', '1% low-fat milk',
       '2% low-fat cottage cheese', '2% lowfat greek yogurt',
       ...
       'yucca root', 'yukon gold', 'yukon gold potatoes', 'yuzu', 'za'atar',
       'zest', 'zesty italian dressing', 'zinfandel', 'ziti', 'zucchini'],
      dtype='object', length=4295)

In [ ]:
test_X = test_data.drop(['index','recipe_id'], axis=1).values
test_X.shape


(44010, 4292)

In [ ]:
prediction = np.argmax(model.predict(test_X), axis=1)

In [ ]:
prediction = prediction.reshape(44010,1)

In [ ]:
res_set = pd.DataFrame.from_records(prediction)
res_set.insert(0,'recipe_id',test_data['recipe_id'])
res_set.columns = ['recipe_id','cuisine']


res_set['cuisine'] = res_set['cuisine'].map(
    { 
        0:'greek', 
        1:'southern_us', 
        2:'filipino', 
        3:'indian', 
        4:'jamaican',
        5:'spanish', 
        6:'italian', 
        7:'mexican', 
        8:'chinese', 
        9:'british', 
        10:'thai',
        11:'vietnamese', 
        12:'cajun_creole', 
        13:'brazilian', 
        14:'french', 
        15:'japanese',
        16:'irish', 
        17:'korean', 
        18:'moroccan', 
        19:'russian'
    }
)



#res_set
res_set.to_csv('submission.csv', index=False)

In [ ]:
res_set

,recipe_id,cuisine
0,6663,french
1,6664,indian
2,6665,filipino
3,6666,southern_us
4,6667,italian
...,...,...
44005,262646,italian
44006,262797,japanese
44007,262815,italian
44008,262850,italian
